<a href="https://colab.research.google.com/github/Moigi/DataCleaning/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spark
!pip install pyspark[sql]
!pandas API on Spark
!pip install pyspark[pandas_on_spark] plotly  # to plot your data, you can install plotly together.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[1]") \
.appName("SparkByExamples.com")\
.getOrCreate()

filePath="/content/drive/MyDrive/BreadBasket_DMS.csv"
df = spark.read.options(header='true', inferSchema='true') \
          .csv(filePath)
df.printSchema()
df.show(4,truncate=False)


In [ ]:
import pyspark
from pyspark.sql import *
spark_app = SparkSession.builder.appName('_').getOrCreate()
# import the Window Function
from pyspark.sql.window import Window
#import the rank from pyspark.sql.functions
from pyspark.sql.functions import rank
#partition the dataframe based on the values in Item column and
#order the rows in each partition based on Time column
partition = Window.partitionBy("Item").orderBy('Time')
#Now mention rank for each row in RANK column
df1=df.withColumn("RANK",rank().over(partition))
df1.show(2)


In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import desc
 
df2 = df1.withColumn("index",monotonically_increasing_id())
df2.orderBy(desc("index")).drop("index").show(2)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
  
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
  
df = spark.read.text("/content/drive/MyDrive/romeo-juliet-pg1777.txt")
  
dfr=df.selectExpr("split(value, ' ') as Text_Data_In_Rows_Using_Text")
dfr.show(4,False)

In [ ]:
from pyspark.sql.functions import col, lower, regexp_replace, split

from pyspark.sql.functions import col, regexp_replace
df = spark.read.text("/content/drive/MyDrive/romeo-juliet-pg1777.txt")
df = df.select(regexp_replace(col('value'), '\n', '').alias("text"))
df.show(5)

In [ ]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="vector")
vector_df = tokenizer.transform(df).select("vector")

vector_df.printSchema()
vector_df.show(10)

In [ ]:
from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Display default list
stopwords[:10]

In [ ]:
# Specify input/output columns
remover.setInputCol("vector")
remover.setOutputCol("vector_no_stopw")

# Transform existing dataframe with the StopWordsRemover
vector_no_stopw_df = remover.transform(vector_df).select("vector_no_stopw")

# Display
vector_no_stopw_df.printSchema()
vector_no_stopw_df.show()

In [ ]:
# Import stemmer library
from nltk.stem.porter import *

# Instantiate stemmer object
stemmer = PorterStemmer()

# Quick test of the stemming function
tokens = ["thanks", "its", "proverbially", "unexpected", "running"]
for t in tokens:
  print(stemmer.stem(t))

In [ ]:
# Create stemmer python function
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        if len(t_stem) > 2:
            out_vec.append(t_stem)       
    return out_vec

# Create user defined function for stemming with return type Array<String>
from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

# Create new df with vectors containing the stemmed tokens 
vector_stemmed_df = (
    vector_no_stopw_df
        .withColumn("vector_stemmed", stemmer_udf("vector_no_stopw"))
        .select("vector_stemmed")
  )

# Rename df and column for clarity
production_df = vector_stemmed_df.select(col("vector_stemmed").alias("unigrams"))

# Display
production_df.printSchema()
production_df.show()

In [ ]:
# Create stemmer python function
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        if len(t_stem) > 2:
            out_vec.append(t_stem)       
    return out_vec

# Create user defined function for stemming with return type Array<String>
from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

# Create new df with vectors containing the stemmed tokens 
vector_stemmed_df = (
    vector_no_stopw_df
        .withColumn("vector_stemmed", stemmer_udf("vector_no_stopw"))
        .select("vector_stemmed")
  )

# Rename df and column for clarity
production_df = vector_stemmed_df.select(col("vector_stemmed").alias("unigrams"))

# Display
production_df.printSchema()
production_df.show()